Как мы успели рассмотреть выше, свёрточный слой нейросети осуществляет преобразование некоторого набора входных карт признаков в новый определенный набор выходных карт признаков. Выбор параметров ядра свёртки, числа входных и выходных каналов, величина и тип расширения (padding) полностью определяет "геометрию" данного преобразования.

На практике, при написании собственных нейронных сетей со свёрточными слоями, мы хотим последовательно пропускать изображение через несколько свёрточных слоёв, передавая полученные на выходе одного свёрточного слоя карты признаков на вход другому. 

Как уже отмечалось выше, свёрточные слои могут принимать на вход карты признаков произвольной ширины и высоты (главное, чтобы ширина и высота карты признаков после расширения (padding) была не меньше размера ядра свёртки) — таким образом, при последовательном соединении свёрточных слоёв остаётся обеспечить согласование числа карт признаков: количество карт признаков на выходе слоя должно равняться числу входных карт признаков следующего после него слоя.

При построении некоторых архитектур CNN (например, для архитектуры [UNet](https://arxiv.org/abs/1505.04597)) важно явно контролировать пространственные размеры всех используемых карт признаков. Чтобы разобраться, как происходит преобразование размеров карт признаков, обратимся к описанию класса [`torch.nn.Conv2d`](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html#conv2d) из библиотеки PyTorch.

`torch.nn.Conv2d` принимает на вход тензор вида $(N, C_{in}, H, W)$, где $N$ нулевом измерении (как всегда) соответствует размерности батча, $C_{in}$ соответствует числу входных карт признаков, а $H$ и $W$ задают пространственные размеры входных карт признаков. На выходе мы получаем тензор вида $(N, C_{out}, H_{out}, W_{out})$, где количество выходных карт признаков $C_{out}$ задаётся при создании экземпляра класса `torch.nn.Conv2d`, а пространственные измерения выходных карт признаков $H_{out}$ и $W_{out}$ вычисляются на основе параметров ядра свёртки и размеров $H$ и $W$ входных карт признаков по формулам:

$$ H_{out} = \lfloor \frac{H + 2 \times \text{padding_h} - \text{dilation_h} \times (\text{kernel_size_h} - 1) -1}{\text{stride_h}}  + 1 \rfloor ,$$
$$ W_{out} = \lfloor \frac{W + 2 \times \text{padding_w} - \text{dilation_w} \times (\text{kernel_size_w} - 1) -1}{\text{stride_w}}  + 1 \rfloor ,$$

где
* `kernel_size_h` и `kernel_size_w` &mdash; количество элементов ядра свёртки по ширине и высоте;
* `padding_h` и `padding_w` &mdash;  параметры расширения выходной карты признаков нулями по ширине и высоте ;
* `stride_h` и `stride_w` &mdash; сдвиг свёртки по ширине и высоте, будет подробно рассмотрено ниже;
* `dilation_h` и `dilation_w` &mdash; расстояние между ядерными элементами (позволяет рассматривать только те пространственные элементы карты признаков, координаты которых кратны величинам сдвига по ширине и высоте; по умолчанию данная величина равна $1$ и в свёртке принимает участие вся карта признаков); С dilated convolutions мы подробнее познакомимся при рассмотрении CNN архитектур, используемых для семантической сегментации.

Интерактивный пример преобразования набора входных карт признаков свёрточным слоем:

In [ ]:
# @title \<code block for conv2d visualization purposes\>

from torch import Tensor

from ipywidgets import interactive
import ipywidgets as widgets

import matplotlib.pyplot as plt
import seaborn as sns


def plot_featuremaps(
    featuremap_tensor: Tensor, title: str = "", v_max: int = None
) -> None:
    n_maps, h, w = featuremap_tensor.shape[1:]
    fig, ax = plt.subplots(
        ncols=n_maps, figsize=(5 * n_maps, 5), sharex=False, sharey=False
    )

    featuremap_tensor = featuremap_tensor.detach()
    if n_maps > 1:
        for id_ax in range(n_maps):
            sns.heatmap(
                featuremap_tensor[0][id_ax],
                ax=ax[id_ax],
                annot=True,
                fmt="0.00f",
                cbar=False,
                vmin=0,
                vmax=v_max,
                linewidths=1,
            )
    else:
        sns.heatmap(
            featuremap_tensor[0][0],
            ax=ax,
            annot=True,
            fmt="0.00f",
            cbar=False,
            vmin=0,
            vmax=v_max,
            linewidths=1,
        )

    fig.suptitle(title)
    plt.show()


def conv2d_example(
    h_in: int = 8,
    w_in: int = 8,
    in_channels: int = 2,
    out_channels: int = 1,
    kernel_size_h: int = 3,
    kernel_size_w: int = 3,
    padding_h: int = 0,
    padding_w: int = 0,
    stride_h: int = 1,
    stride_w: int = 1,
    dilation_h: int = 1,
    dilation_w: int = 1,
) -> None:
    """
    This function generates an example of the input
    feature maps tensor, passes it to a two-dimensional
    convolution with the specified parameters and
    unit weights, and then returns the resulting
    feature maps output tensor.
    """

    dummy_input = torch.tensor(
        [[list(range(h_in))] * w_in] * in_channels, dtype=torch.float
    ).unsqueeze(
        0
    )  # Let's create an example input tensor like $(1, C_{in}, H. W)$

    conv_layer = nn.Conv2d(
        in_channels=in_channels,
        out_channels=out_channels,
        kernel_size=(kernel_size_h, kernel_size_w),
        padding=(padding_h, padding_w),
        stride=(stride_h, stride_w),
        dilation=(dilation_h, dilation_w),
        bias=False,
    )  # Creating an instance of the 2D convolution class with the given parameters.

    conv_layer.weight = nn.Parameter(
        torch.ones_like(conv_layer.weight)
    )  # Replace random weights to ones
    output = conv_layer(dummy_input)
    print("\n\nExample of input feature maps:")
    plot_featuremaps(
        dummy_input, f"input featuremap tensor\nshape = {dummy_input.shape}", v_max=h_in
    )

    print(f"\n\nconv kernel shape is {conv_layer.weight.shape}\n\n")

    print("Example of output feature maps:")
    plot_featuremaps(output, f"output featuremap tensor\nshape = {output.shape}")


conv2d_example_interactive = interactive(
    conv2d_example,
    h_in=widgets.IntSlider(min=8, max=32, step=1, value=8),
    w_in=widgets.IntSlider(min=8, max=32, step=1, value=8),
    in_channels=widgets.IntSlider(min=1, max=4, step=1, value=3),
    out_channels=widgets.IntSlider(min=1, max=4, step=1, value=2),
    kernel_size_h=widgets.IntSlider(min=1, max=8, step=1, value=3),
    kernel_size_w=widgets.IntSlider(min=1, max=8, step=1, value=3),
    padding_h=widgets.IntSlider(min=0, max=8, step=1, value=0),
    padding_w=widgets.IntSlider(min=0, max=8, step=1, value=0),
    stride_h=widgets.IntSlider(min=1, max=8, step=1, value=1),
    stride_w=widgets.IntSlider(min=1, max=8, step=1, value=1),
    dilation_h=widgets.IntSlider(min=1, max=8, step=1, value=1),
    dilation_w=widgets.IntSlider(min=1, max=8, step=1, value=1),
)

In [ ]:
display(conv2d_example_interactive)

interactive(children=(IntSlider(value=8, description='h_in', max=32, min=8), IntSlider(value=8, description='w…

Мы успели довольно подробно рассмотреть свёрточный слой и заметить большое количество его сходств с полносвязным слоем (перцептроном). Теперь давайте попробуем сравнить эти два слоя по требуемому количеству параметров и операций умножения.  

В первую очередь давайте определимся с размерами входного и получаемого тензоров. Пусть на вход передаётся $C_{in}\times H_{in}\times W_{in}$. На выходе пусть будет $K$ нейронов для полносвязного слоя и $C_{out}\times H_{out} \times W_{out}$ для свёрточного слоя (фильтр имеет размер $C_{in} \times F_1 \times F_2$). $F_1$ и $F_2$ - размер ядра свёртки по высоте и ширине, соответственно.

Для простоты расчётов давайте примем, что шаг фильтра равен $1$ как по горизонтали, так и по вертикали. В таком случае, $H_{out} = H_{in} - F_1 + 1$, а $W_{out} = W_{in} - F_2 + 1$.

##### Количество параметров:  
***Полносвязный слой:***  
Данный слой требует по параметру (весу) для всех связей между входными и выходными нейронами, то есть $C_{in} \cdot H_{in} \cdot W_{in} \cdot K$. Помимо этого, каждый из выходных нейронов имеет свободный член, общее количество которых $K$. Итого, количество обучаемых параметров: $(C_{in} \cdot H_{in} \cdot W_{in} + 1) \cdot K$.

***Свёрточный слой:***  
Для свёрточного слоя параметры связаны лишь с ядрами свёртки: внутри каждого ядра находятся $C_{in} \cdot F_1 \cdot F_2$ параметров, общее их количество &mdash; $C_{out}$. Помимо этого, каждое из ядер имеет свой собственный свободный член, поэтому общее количество обучаемых параметров: $(C_{in} \cdot F_1 \cdot F_2 + 1) \cdot C_{out}$.

***Сравнение количества параметров:***  
Поскольку количество свободных членов мало относительно количества весов, мы опустим их в этих расчётах. 
$$Comp_{param} = \frac{C_{in} \cdot H_{in} \cdot W_{in} \cdot K}{C_{in} \cdot F_1 \cdot F_2 \cdot C_{out}} = \frac{H_{in} \cdot W_{in} \cdot K}{F_1 \cdot F_2 \cdot C_{out}}.$$ 

##### Количество умножений: 
***Полносвязный слой:***  
В данном слое каждый вес используется лишь один раз, в результате общее количество умножений: $$C_{in} \cdot H_{in} \cdot W_{in} \cdot K$$.  

***Свёрточный слой:***
Заметим, что, в отличие от перцептрона, свёрточная нейронная сеть использует каждый вес несколько раз (при подсчёте каждого из элементов карты активации). Размер карты активаций: $H_{out} \times W_{out}$. В итоге оказывается, что количество операций умножения равно: $$C_{in} \cdot F_1 \cdot F_2 \cdot C_{out} \cdot H_{out} \cdot W_{out}$$.

***Сравнение количества умножений:***
$$Comp_{mult} = \frac{C_{in} \cdot H_{in} \cdot W_{in} \cdot K}{C_{in} \cdot F_1 \cdot F_2 \cdot C_{out} \cdot H_{out} \cdot W_{out}} = \frac{H_{in} \cdot W_{in} \cdot K}{F_1 \cdot F_2 \cdot C_{out} \cdot H_{out} \cdot W_{out}} = \frac{Comp_{param}}{H_{out} \cdot W_{out}}.$$